<a href="https://colab.research.google.com/github/dvircohen0/Machine-Learning-Algorithms-From-Scratch/blob/main/spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!wget http://openclassroom.stanford.edu/MainFolder/courses/MachineLearning/exercises/ex6materials/ex6DataEmails.zip
!unzip /content/ex6DataEmails.zip

--2021-04-30 14:03:23--  http://openclassroom.stanford.edu/MainFolder/courses/MachineLearning/exercises/ex6materials/ex6DataEmails.zip
Resolving openclassroom.stanford.edu (openclassroom.stanford.edu)... 171.64.68.10
Connecting to openclassroom.stanford.edu (openclassroom.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 939991 (918K) [application/zip]
Saving to: ‘ex6DataEmails.zip’

ex6DataEmails.zip   100%[===================>] 917.96K   371KB/s    in 2.5s    

2021-04-30 14:03:25 (371 KB/s) - ‘ex6DataEmails.zip’ saved [939991/939991]



In [54]:
import os

def file_to_word_list(file_path):
  a_file = open(file_path,'r')
  for line in a_file:
    stripped = line.strip()
    line_list = stripped.split()
  return line_list

spam_path = '/content/spam-train'
spam_files = os.listdir(spam_path)
list_of_spam =[]
for file in spam_files:
  full_path = os.path.join(spam_path,file)
  list_of_spam.append(file_to_word_list(full_path))


ham_path = '/content/nonspam-train'
ham_files = os.listdir(ham_path)
list_of_ham =[]
for file in ham_files:
  full_path = os.path.join(ham_path,file)
  list_of_ham.append(file_to_word_list(full_path))


In [49]:
class Vocabulary:
  def __init__(self):
    self.word2index={}
    self.word2count={}
    self.index2word={}
    self.wordprob={}
    self.num_words = 0
    self.num_sent = 0

  def add_word(self, word):
      if word not in self.word2index:
          # First entry of word into vocabulary
          self.word2index[word] = self.num_words
          self.word2count[word] = 1
          self.index2word[self.num_words] = word
          self.num_words += 1
      else:
          # Word exists; increase word count
          self.word2count[word] += 1

  def add_sentence(self, sentence):
    self.num_sent +=1
    for word in sentence:
      self.add_word(word)

  def word_probs(self):
    for word in self.word2count:
      self.wordprob[word] = self.word2count[word]/self.num_words


In [55]:
spam = Vocabulary()
for word_list in list_of_spam:
  spam.add_sentence(map(str.lower,word_list))
spam.word_probs()


ham = Vocabulary()
for word_list in list_of_ham:
  ham.add_sentence(map(str.lower,word_list))
ham.word_probs()

In [56]:
ham.wordprob

{'re': 0.004595913172594216,
 'german': 0.007070635650144948,
 'affricate': 0.00014141271300289896,
 'regard': 0.0021211906950434844,
 'roger': 0.0006363572085130453,
 'lass': 7.070635650144948e-05,
 's': 0.046736901647458105,
 'comment': 0.0022626034080463833,
 'ts': 0.0003535317825072474,
 'v': 0.011595842466237715,
 't': 0.005020151311602913,
 'c': 0.011171604327229018,
 'put': 0.0014141271300289896,
 'd': 0.01484833486530439,
 'pronunciation': 0.0012020080605246411,
 'pertain': 0.0002828254260057979,
 'many': 0.006717103867637701,
 'american': 0.0036060241815739236,
 'english': 0.02206038322845224,
 'speaker': 0.010040302623205826,
 'm': 0.014070564943788447,
 'alone': 0.00021211906950434844,
 'something': 0.001767658912536237,
 'glottal': 7.070635650144948e-05,
 'stop': 0.001484833486530439,
 'esh': 7.070635650144948e-05,
 'raise': 0.002191897051544934,
 'case': 0.013858445874284099,
 'indicate': 0.00275754790355653,
 'difference': 0.001626246199533338,
 'one': 0.01979777982040585

In [34]:
class NaiveBayes:

  def fit(self, X,y):
    n_samples, n_features = X.shape
    self._classes = np.unique(y)
    n_classes = len(self._classes)

    self._mean = np.zeros((n_classes,n_features), dtype=np.float64)
    self._var = np.zeros((n_classes,n_features), dtype=np.float64)
    self._priors = np.zeros(n_classes, dtype=np.float64)

    for c in self._classes:
      X_c = X[c==y]
      self._mean[c,:] = X_c.mean(axis=0)
      self._var[c,:] = X_c.var(axis=0)
      self._priors[c,:] = X_c.shape[0] / float(n_samples)

  def predict(self,X):
    y_pred = [self._predict(x) for x in X]
    return y_pred


  def _predict(self, x):
    posteriors = []
    for idx, c in enumerate(self._classes):
      ptior = np.log(self._priors[idx])
      cond_prob = np.sum(np.log(self.dist_prob(idx,x)))
      posterior =ptior +  cond_prob
      posteriors.append(posterior)
    return self._classes[np.argmax(posteriors)]

  def dist_prob(self, class_idx,x):
    mean = self._mean[class_idx]
    var = self._var[class_idx]
    return  (np.exp(-(x-mean)**2 / (2* var)))/(np.sqrt(2*np.pi * var))